In [1]:
import json
from pathlib import Path

In [2]:
# Read passed universe JSON
uni_path = Path("../analyses/current_universe.json")
with open(uni_path) as f:
    universe = json.load(f)

universe

FileNotFoundError: [Errno 2] No such file or directory: '../analyses/current_universe.json'

In [ ]:
# Data
from src.data import load_data, filter_data
df = load_data()
df = filter_data(df, exclude=universe['exclude_subgroups'])

In [ ]:
# Features
from src.features import build_features
X, y = build_features(df, universe['feature_set'])

In [ ]:
# Model Training
from src.model import train_model, apply_threshold
model = train_model(X, y, universe['model'])
y_pred, thr = apply_threshold(model, X, policy=universe['threshold_policy'])

In [ ]:
# Evaluation
from src.metrics import compute_metrics
metrics = compute_metrics(y, y_pred)
print(metrics)

In [ ]:
# Save
import pandas as pd
pd.DataFrame([metrics]).assign(**universe).to_csv(
    f"../analyses/metrics_universe_{universe['feature_set']}_{universe['model']}.csv",
    index=False
)